# EnvisionBOX Summer School 2025, Day 4: Bringing all together
## Data Analysis

At this point, we have one file with all signals and annotations that are relevant for us per each trial. Now the ways will diverge depending on what questions we want to ask.

Are we still interested in the temporal characteristics of the **whole** signal or its shape, maybe in relation to other trials? Then we will probably continue working with the whole timeseries.

Do we need to get information from the signal that will represent the trial's temporal and/or other aspect? Then we most likely can collect the information into a **flat** dataframe, where we no longer have the time dimension, but rather a set of features that describe the trial.

## Timeseries analysis

> How similar two signals are?			Dynamic Time Warping, Euclidian distance
> What is the character of a signal?		local vs. global character, descriptives, peaks> What feature most contributes to …?
        PCA, eXtreme Gradient Boosting> What is the complexity of a signal?		PCA

Possible directions of timeseries analysis include:

- **synchrony analysis within person**: https://www.envisionbox.org/embedded_CrossWavelet_SG_synchrony.html
- **synchrony analysis between persons**: https://www.envisionbox.org/embedded_interpersonal_synchrony.html
- **non-linear time series analysis**: https://wimpouw.github.io/InterPerDynPipeline/
- **similarity analysis**: https://www.envisionbox.org/embedded_Gesture_kinematic_spaces.html

## Flat data analysis

What we want to do now is create one master dataframe with some information for each trial. What information again depends on the questions we want to ask. You could keep it very simple and extract one or few features from the signals because this is what you need, or you want to be a bit megalomaniac or maybe you are not necessarily sure what feature would capture some difference between trials, or participants, or conditions, so you decide to extract a lot of features and work with multi-dimensional matrix of features.

Things you could collect when it comes to gestural & spoken behaviour include:

- basic statistics (mean, std, ...)
- gesture/vocal space
- complexity measures ~ intermittency, entropy, temporal variability, number of principal components (via PCA), ...

And many more...

In [1]:
# Import packages
import os
import glob
import pandas as pd
from scipy.signal import find_peaks
import numpy as np
import antropy as ent
import matplotlib.pyplot as plt
#import umap
#import seaborn as sns

curfolder = os.getcwd()

# Here we store our merged final timeseries data
datafolder = os.path.join(curfolder, 'TS_annotated')
filestotrack = glob.glob(os.path.join(datafolder, '*.csv'))

In [2]:
# Function to calculate stats for a feature
def get_statistics(cols, df, subdf, dictionary):
    for col in cols:
        Gmean = subdf[col].mean()   # General mean
        Gstd = subdf[col].std()     # General std

        # If the col is 2nd derivative of position data, joint kinematics, or moments, we will calculate both positive and negative peaks (by reversing the sign of the data)
        if 'acc' in col:
            pospeaks, _ = find_peaks(subdf[col])
            pospeaks_values = df.loc[pospeaks, col]
            pospeaks_times = df.loc[pospeaks, "Time"].tolist()

            negpeaks, _ = find_peaks(-subdf[col])
            negpeaks_values = df.loc[negpeaks, col]
            negpeaks_times = df.loc[negpeaks, "Time"].tolist()

            negpeak_mean = negpeaks_values.mean()
            negpeak_std = negpeaks_values.std()
            negpeak_n = len(negpeaks_values)

        else:    
            pospeaks, _ = find_peaks(subdf[col])

            pospeaks_values = df.loc[pospeaks, col] # Peak values
            pospeaks_times = df.loc[pospeaks, "Time"].tolist() # Peak time

            # Plot if needed
            # peak_times = subdf.iloc[pospeaks]['time']
            # peak_values = subdf.iloc[pospeaks][col]
            # plt.plot(subdf['time'], subdf[col], label="Signal")
            # plt.scatter(peak_times, peak_values, color='r', label="Pos Peaks", marker="x")
            # plt.title(col)
            # plt.legend()
            # plt.show()

            pospeak_mean = pospeaks_values.mean() # Peak mean
            pospeak_std = pospeaks_values.std()   # Peak std
            pospeak_n = len(pospeaks_values)      # Number of peaks

            negpeak_mean = None
            negpeak_std = None
            negpeak_n = None
            negpeaks_times = None

        integral = np.trapz(subdf[col]) # Integral
        range = subdf[col].max() - subdf[col].min() # Range
        
        # Save all in dictionary
        dictionary[col] = [Gmean, Gstd, pospeak_mean, pospeak_std, pospeak_n, pospeaks_times, negpeak_mean, negpeak_std, negpeak_n, negpeaks_times, integral, range]

    return dictionary

# Function to adapt row
def adapt_row(row_to_process):
    for col in row_to_process.columns:
        # Calculate for all expcet some already calculated measures
        if 'inter' not in col and 'sampen' not in col and 'bbmv' not in col and 'duration' not in col and 'arms_submov' not in col and 'movid' not in col:
            row_to_process[col + '_Gmean'] = row_to_process[col].apply(lambda x: x[0])
            row_to_process[col + '_Gstd'] = row_to_process[col].apply(lambda x: x[1])
            row_to_process[col + '_pospeak_mean'] = row_to_process[col].apply(lambda x: x[2])
            row_to_process[col + '_pospeak_std'] = row_to_process[col].apply(lambda x: x[3])
            row_to_process[col + '_pospeak_n'] = row_to_process[col].apply(lambda x: x[4])
            row_to_process[col + '_pospeak_times'] = row_to_process[col].apply(lambda x: x[5])
            row_to_process[col + '_negpeak_mean'] = row_to_process[col].apply(lambda x: x[6])
            row_to_process[col + '_negpeak_std'] = row_to_process[col].apply(lambda x: x[7])
            row_to_process[col + '_negpeak_n'] = row_to_process[col].apply(lambda x: x[8])
            row_to_process[col + '_negpeak_times'] = row_to_process[col].apply(lambda x: x[9])
            row_to_process[col + '_integral'] = row_to_process[col].apply(lambda x: x[10])
            row_to_process[col + '_range'] = row_to_process[col].apply(lambda x: x[11])

    # Now keep only this newly created cols
    row_final = row_to_process[[col for col in row_to_process.columns if any(x in col for x in ['Gmean', 'Gstd', 'pospeak_mean', 'pospeak_std', 'pospeak_n', 'sampen', 'inter', 'integral', 'pospeak_times', 'bbmv', 'range', 'duration', 'negpeak_mean', 'negpeak_std', 'negpeak_n', 'negpeak_times', 'arms_submov', 'movid'])]]

    # Get rid of cols with NaNs
    row_final = row_final.dropna(axis=1, how='all')
    
    return row_final

Now you could add any other potential functions that take in a timeseries of sound/movement and return a single value or a set of features.

In [3]:
# Function to calcuate bounding box of movement volume     
def get_bbmv(df, group, kp_dict):
    coordinates = [col for col in df.columns if any(x in col for x in ['_x', '_y', '_z'])]

    # Prepare columns that belong to a group (e.g., arm)
    kp = kp_dict[group]
    colstoBBMV = [col for col in coordinates if any(x in col for x in kp)]

    # Keep only unique names without coordinates
    kincols = list(set([col.split('_')[0] for col in colstoBBMV]))

    bbmvs = {}
    for col in kincols:
        # Span of x, y, z
        x_span = df[col + '_x'].max() - df[col + '_x'].min()
        y_span = df[col + '_y'].max() - df[col + '_y'].min()
        z_span = df[col + '_z'].max() - df[col + '_z'].min()

        # Calculate BBMV
        bbmv = x_span * y_span * z_span
        bbmvs[col] = bbmv
        
    # Get the sum for the whole group
    bbmv_sum = sum(bbmvs.values())

    # Natural logarithm
    bbmv_sum = np.log(bbmv_sum) 

    return bbmv_sum

In [4]:
# Function to calculate intermittency 
def get_intermittency(jerk_values, speed_values):
    """Calculate the dimensionless smoothness measure using precomputed smoothed jerk and speed."""
    smoothed_jerk = jerk_values
    speed = speed_values
    
    if not np.all(speed == 0):
        integrated_squared_jerk = np.sum(smoothed_jerk ** 2)
        max_squared_speed = np.max(speed ** 2)
        D3 = len(speed) ** 3
        jerk_dimensionless = integrated_squared_jerk * (D3 / max_squared_speed)
        smoothness = jerk_dimensionless
    else:
        smoothness = np.nan

    return smoothness

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Initialize the dataframe
features_df = pd.DataFrame()

##########################
##### preparations #######
##########################

# These are groups of keypoints belonging to one body part
kp_arms = ['RWrist', 'RElbow', 'RShoulder', 'LWrist', 'LElbow', 'LShoulder']
kp_lower = ['RAnkle', 'RKnee', 'LAnkle', 'LKnee']
kp_legs = ['RAnkle', 'RKnee', 'LAnkle', 'LKnee']
kp_head = ['Head']
kp_keys = {'arm': kp_arms, 'lowerbody': kp_lower, 'leg': kp_legs, 'head': kp_head}

##########################
####### main loop ########
##########################

for file in filestotrack:
    print('working on file: ', file)
    df = pd.read_csv(file)

    # Add gesture annotation ID, such that everytime a value changes, we increment the ID
    df["GestureID"] = (df["Gesture"] != df["Gesture"].shift()).cumsum()

    # chunk the df in chunks according to GestureID
    df_chunks = [group for _, group in df.groupby("GestureID")]

    # into movchunks add all chunks where Gesture has value Move
    movchunks = [group for group in df_chunks if group["Gesture"].values[0] == "Move"]
    geschunks = [group for group in df_chunks if group["Gesture"].values[0] == "Gesture"]

    choi = movchunks + geschunks

    i = 1 
    for ch in choi:
        
        subdf = ch
        sumfeat = {}

        # if Gesture value is Gesture, mov_id == gesture, else mov
        if subdf["Gesture"].values[0] == "Gesture":
            movid = f'gesture_{i}'
        else:
            movid = f'mov_{i}'

        if subdf.shape[0] == 0:
            print("Empty chunk, skipping")
            # TODO-ad zero values
            continue

        else:
            colstosum = [col for col in subdf.columns if 'Time' not in col and 'file' not in col and 'audio' not in col and 'Gesture' not in col and 'GestureID' not in col]
            # Get stats
            sumfeat = get_statistics(colstosum, df, subdf, sumfeat)
            # Get duration
            duration = subdf["Time"].iloc[-1] - subdf["Time"].iloc[0]
            sumfeat['duration'] = duration
            # Get intermittency for RWrist
            if 'RWrist_speed' in subdf.columns and 'RWrist_jerk' in subdf.columns:
                speed = subdf['RWrist_speed'].values
                jerk = subdf['RWrist_jerk'].values
                intermittency = get_intermittency(jerk, speed)
                intermittency = np.log(intermittency)
                sumfeat['RWrist_inter'] = intermittency

            else:
                print("RWrist speed or jerk not found, skipping intermittency calculation")
                sumfeat['RWrist_inter'] = np.nan
                
            # Get sample entropy for RWrist
            if 'RWrist_speed' in subdf.columns:
                sampEn = ent.sample_entropy(subdf['RWrist_speed'].values, 2, 0.2 * np.std(subdf['RWrist_speed'].values))
                sumfeat['RWrist_sampen'] = sampEn

            # Get BBMV for each group
            for group in kp_keys.keys():
                bbmv_sum = get_bbmv(subdf, group, kp_keys)
                sumfeat[group + '_bbmv'] = bbmv_sum

            # Number of submovements
            submovcols = [col for col in subdf.columns if "Wrist" in col and 'speed' in col]
            arms_submov = []
  
            for col in submovcols:
                # Calculate the number of peaks that delimit unique submovements
                peaks, _ = find_peaks(subdf[col])
                arms_submov.append(len(peaks))

            sumfeat['arms_submov'] = np.sum(arms_submov)
            sumfeat['movid'] = movid

            # Make a row
            sumfeat_row = pd.DataFrame({key: [value] for key, value in sumfeat.items()})
            sumfeat_row = adapt_row(sumfeat_row)
            
            # Concat the new row to the features_df
            if features_df.empty:
                features_df = sumfeat_row
            else:
                features_df = pd.concat([features_df, sumfeat_row], ignore_index=True)

        
        i += 1
                

# Get rid of columns that have only NA values
#features_df = features_df.dropna(axis=1, how='all')

# Round all num cols to 3
#numcols = features_df.select_dtypes(include=[np.number]).columns
#features_df[numcols] = features_df[numcols].round(3)




working on file:  c:\Users\Sarka Kadava\Documents\Github\envisionBOX_SummerschoolAmsterdam2025\Day4_MultimodalMerging\TS_annotated\annotated_femalemonologue2_t3_0-619.csv


In [7]:
features_df

,duration,RWrist_inter,RWrist_sampen,arm_bbmv,lowerbody_bbmv,leg_bbmv,head_bbmv,arms_submov,movid,RHip_x_Gmean,...,RHip_acc_pospeak_std,RHip_jerk_pospeak_std,Nose_speed_pospeak_std,Nose_acc_pospeak_std,Nose_acc_negpeak_std,Nose_jerk_pospeak_std,LElbow_jerk_pospeak_std,LShoulder_jerk_pospeak_std,LWrist_speed_pospeak_std,LWrist_acc_pospeak_std
0,200.000,10.565,0.021,2.902,3.119,3.119,-2.860,0,mov_1,-255.851,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,600.000,13.045,0.034,6.297,5.836,5.836,2.342,3,mov_2,-254.909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,520.000,11.455,0.025,6.364,6.099,6.099,1.547,2,mov_3,-267.857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,920.000,13.156,0.116,6.918,5.358,5.358,5.643,7,mov_4,-275.858,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,280.000,8.988,0.005,9.115,-0.013,-0.013,-5.270,1,gesture_5,-292.472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,840.000,15.171,0.028,9.906,8.816,8.816,7.273,2,gesture_6,-281.960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,280.000,9.645,0.018,4.705,4.156,4.156,-1.545,1,gesture_7,-256.477,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,480.000,12.136,0.021,8.302,5.477,5.477,4.593,2,gesture_8,-284.522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2600.000,18.895,0.020,12.952,12.020,12.020,10.009,17,gesture_9,-327.991,...,0.003,0.0,0.004,0.004,0.001,0.0,0.0,0.0,0.346,0.346
9,1126.667,17.060,0.030,10.855,4.781,4.781,2.877,2,gesture_10,-284.471,...,NaN,0.0,0.003,0.003,0.001,NaN,NaN,NaN,NaN,NaN


So now we end up with nice dataframe that is ready to be visualized, inspected, analyzed...

Next steps

- UMAP
 non-linear method
 preserves both local and global structure
 preserves local and gloval relationships (overal shape + clusters)
 output: coordinates/embeddings 
 stochastic ~ different results every time, due to randomizations
 cluster sizes and distances are not meaningful!!!
- t-SNE
 non-linear method
 preserves local structure (neighborhoods)
 focuses on local relationships
 quite slow
 output: coordinates/embeddings 
 stochastic ~ different results every time, due to randomizations
cluster sizes and distances are not meaningful!!!

- Dashboard
- PCA
 linear method (linear transformation)
 focuses on maximizing variance (global structure)
 preserves variance (overall spread)
 output: principal components (PCs)
 deterministic ~ same results every time
 (or see non-linear PCA)
- eXtreme Gradient Boosting
- Euclidian distance / trial embeddings